In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Testing a CNN on MNIST  
For https://www.kaggle.com/c/digit-recognizer
Old but still works. Time to update to the latest tensoflow and keras API  
scores 0.99117

### Imports required

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Have a look at the data and prepare it.

In [3]:
X = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X.shape

In [4]:
X.head()

In [5]:
y = X['label']

In [6]:
X.drop(['label'], axis=1, inplace=True)

In [7]:
# normalise the data by dividing by the max
X /= 255.

In [8]:
# convert X to an array
X = np.array(X)
# reshape to be [samples][width][height][channels]
Xr = X.reshape(X.shape[0], 28, 28, 1)

# normalise the data divide by the highest value 
# Xr /= 255 works in jupyter but not here? moved above

In [9]:
Xr.shape

In [10]:
# convert class vectors to binary class matrices
num_classes = 10
y = to_categorical(y, num_classes)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(Xr, y, test_size=0.1, random_state=42, shuffle=True)

In [12]:
red_lr = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, mode='max' )

es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=8)

model = Sequential()

model.add(Conv2D(64, input_shape=(28, 28, 1) , kernel_size=(7,7), activation='elu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='elu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(300, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Train the model
model_his = model.fit(X_train, y_train, 
                      batch_size=50,
                      epochs=50, 
                      callbacks=[es, red_lr],
                      validation_data=(X_val, y_val) 
                      )

# Plot train vs test accuracy per epoch
plt.figure()
# Use the history metrics
plt.plot(model_his.history['acc'])
plt.plot(model_his.history['val_acc'])
# Make it pretty
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
# the highest accuracy score for the adding another hidden layer
print('\nThe highest test score', max(model_his.history['val_acc']).round(3))
plt.show()

In [13]:
X_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
X_test.shape

In [14]:
# normalise the data by dividing by the max
X_test /= 255.
# convert X to an array
X_test = np.array(X_test)
# reshape to be [samples][width][height][channels]
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)



In [15]:
#Train the model
model.fit(Xr, y, 
          batch_size=50,
          epochs=25, 
          callbacks=[red_lr] ) # need to shedule the learning rate reduction

preds = model.predict(X_test)

In [16]:
preds[0]

In [17]:
# convert back from a categorical array to an array of the digits predicted and expected
num_preds = np.argmax(preds, axis=1)

In [18]:
sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [19]:
sub.head()

In [20]:
sub['Label'] = num_preds

In [21]:
sub.head()

In [22]:
sub.to_csv('./submission.csv', index=False)